# API: Museum

В рамках дополнительного проекта попробуем освоить API, позволяющий получать данные о музейных коллекциях. Это задание может пригодиться для большего понимания того, какими способами можно собирать и обрабатывать данные, например, для собственного проекта или другой рабочей задачи.

Документация: https://metmuseum.github.io

- Объекты (Objects)
- Объект (Object) – данные об объекте (например, о картине)
- Департаменты (Departments) –  данные о департаменте/разделе (например, "Современное искусство") 
- Поиск (Search) – используется для поиска

In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Сам запрос к API должен состоять из нескольких параметров. Пример обращения к поиску (Search):

In [2]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=QUERY')
res = r.json()
# QUERY – ключевое слово

Найти все произведения, связанные с подсолнухами. Выполните поиск по слову "sunflowers". 
Внимательно посмотрите на формат ответа и сохраните id полученных объектов в список sunflower_ids.

In [3]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
res = r.json()
sunflower_ids = res['objectIDs']

In [4]:
# url = '<https://collectionapi.metmuseum.org/public/collection/v1/search?q=>'
# IDs = 'sunflowers'
# r = requests.get(url+IDs)
# res = r.json()
# Где, IDsэто ключевое слово, в нашем случае sunflowers(подсолнух)

In [5]:
437329 in sunflower_ids

True

In [6]:
16822570 in sunflower_ids

False

In [7]:
436524 in sunflower_ids


True

In [9]:
# Другой вариант:
search_ids = [436524, 2032, 20149, 16822570, 437329]  # записываем варианты id

In [10]:
# для каждого id проверяем, есть ли он в sunflower_ids; выводим элемент, если условие выполнено
[i for i in search_ids if i in sunflower_ids]

[436524, 2032, 437329]

Пример текста запроса для получения информации об одном объекте (Object):

In [11]:
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/OBJECT_ID')
obj_test = r_test.json()
# где OBJECT_ID – id объекта.
# Выполните запрос для объекта 437980. Для проверки соотнесите полученные параметры с их значениями:

In [12]:
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')
obj_test = r_test.json()

In [13]:
print(obj_test['culture'])
print(obj_test['title'])
print(obj_test['department'])
print(obj_test['artistDisplayName'])
print(obj_test['objectName'])
print(obj_test['objectID'])



Cypresses
European Paintings
Vincent van Gogh
Painting
437980


Напишите цикл, с помощью которого будет собрана информация об объектах, которые были получены на 2 шаге. 
Результаты запишите в датафрейм, а именно – следующие параметры:
- objectID – id объекта
- title – название
- artistDisplayName – автор
- department – департамент
- objectBeginDate – дата (начало)
- objectEndDate – дата (конец)
- period – название периода
- objectName – название/категория объекта
- culture – культура

Написать запрос к API, вместо object_id подставить id из sunflower_ids и сформировать строчку в датафрейме для объекта

In [14]:
sunflower_df = pd.DataFrame() #создали пустой df
for i in sunflower_ids:
    try:
        r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(i))
        obj = r.json()
   
    
        df = pd.json_normalize(obj)
        df = df[['objectID',
                                 'title',
                                 'artistDisplayName',
                                 'department',
                                 'objectBeginDate',
                                 'objectEndDate',
                                 'period',
                                 'objectName',
                                 'culture']]
    except:
        continue

    try:
        sunflower_df = pd.concat([sunflower_df, df])
    except:
        continue



In [15]:
sunflower_df.query('culture == "Japan"')

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,57922,Incense box,,Asian Art,1700,1799,Edo period (1615–1868),Incense box,Japan


In [16]:
sunflower_df.shape

(88, 9)

Теперь попробуем построить более сложный запрос с помощью секции "Search"! 
На этот раз параметры необходимо передать через "&", например, departmentId (id департамента – 1) и q (ключевое слово – cat):

In [17]:
# r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=1&q=cat')
# cats = r.json()

Выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишите id объектов в cat_ids. 
Используйте цикл из шага 4, чтобы получить данные о первой 1000 объектов из списка.

выполнение цикла может занять некоторое время (~5 минут), поэтому проверять его работоспособность лучше на маленькой выборке!

In [18]:
r_cats = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r_cats.json()
cat_ids = cats['objectIDs']

In [19]:
# for i in cat_ids[0:10]:
    
#     try:
#         r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(i))
#         obj = r.json()
   
    
#         dff = pd.json_normalize(obj)
#         dff = dff[['objectID', 'title', 'period', 'objectName']]
    
#     except:
#         continue
        
#     try: 
#         cat_df = pd.concat([cat_df, df])
                      
#     except:
#         continue

# cat_df

In [20]:
cat_df = pd.DataFrame()
for i in cat_ids[0:1000]:
    r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(i))
    obj = r.json()    
    dff = pd.json_normalize(obj)
    dff = dff[['objectID', 'period']]
    try: 
        cat_df = pd.concat([cat_df, dff])
                      
    except:
        continue

cat_df.period.value_counts().head()

Edo period (1615–1868)         275
                               144
Qing dynasty (1644–1911)        62
Ming dynasty (1368–1644)        42
Kamakura period (1185–1333)     32
Name: period, dtype: int64

Другой вариант:

In [21]:
# собираем датафрейм для результатов поиска 
object_df = pd.DataFrame()

for object_id in cat_ids[:1000]:  # берем только 1000 объектов, как указано в задании
    # информация об объекте
    r = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{object_id}')
    obj_tmp = r.json()

    # формируем строчку для объекта
    try:
        obj_tmp_df = pd.DataFrame({'objectID' : obj_tmp['objectID'],
                                 'title' : [obj_tmp['title']],
                                 'artistDisplayName': [obj_tmp['artistDisplayName']],
                                 'department' : [obj_tmp['department']], 
                                 'objectBeginDate' : [obj_tmp['objectBeginDate']], 
                                 'objectEndDate' : [obj_tmp['objectEndDate']], 
                                 'objectName' : [obj_tmp['objectName']], 
                                 'culture': [obj_tmp['culture']],
                                 'period': [obj_tmp['period']]
                            })
        # приклеиваем
        object_df = pd.concat([object_df, obj_tmp_df])
    except KeyError:
        continue
